<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/PLAN604_0av1_US_Census_Geography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding US Census Geography

This program focuses on obtaining US Census Geography shapefiles.


In [ ]:
# Install packages not included in Google Colab (-q = option quiet)
!pip -q install geopandas
# Import packages
import sys # For checking version of python for replication
import os   # For saving output to path
import pandas as pd # For reading, writing and wrangling data
import geopandas as gpd # For creating maps
import folium as fm # For creating the final HTML map complete with OSM (Open Street Map) basemap

from google.colab import files  # Required to automatedownload final files 

Using the wget command download the cartographic boundaries files from the US Census website.

The cartographic files come in three levels of resolution. The "smallest" file size will be the lowest resolution - 20m = 1:20,000,000. For smaller geographies the 5m = 1:5,000,000 resolution and the 500k = 1:500,000 resolutions provide more details.

In [ ]:
# Program Prepare Geography combines steps needed to prepare files for mapping
def prepare_geography(year,state,entity,resolution):
  filename = f'cb_{year}_{state}_{entity}_{resolution}'

  !wget -q https://www2.census.gov/geo/tiger/GENZ2019/shp/{filename}.zip

  !unzip -o -q {filename}.zip
  gdf = gpd.read_file(f'{filename}.shp')
  gdf = gdf.to_crs(epsg=4326)
  
  return gdf

## Create a list of the geographies to be prepared
The list includes the geographic entity and the resolution.

In [ ]:
state = '48'
entities = [['us','nation','5m'],
            ['us','region','5m'],
            ['us','division','5m'],
            ['us','state','5m'],
            ['us','county','5m'],
            ['us','cbsa','20m'],
            ['us','aiannh','500k'],
            ['us','ua10','500k'],
            [state,'place','500k'],
            [state,'tract','500k'],
            [state,'bg','500k']]

In [ ]:
geographies = {}
for entity in entities:
    geographies[entity[1]] = prepare_geography('2019',entity[0],entity[1],entity[2])

    # Approximate measure of sq km area based on Equal Area Cylindrical projection (cea)
    # https://proj.org/operations/projections/cea.html
    geographies[entity[1]]= geographies[entity[1]].to_crs({'proj':'cea'})
    geographies[entity[1]]["areasqkm"] = geographies[entity[1]]['geometry'].area/ 10**6

    # Set geography to EPSG 4326
    # EPSG: 4326 uses a coordinate system  (Lat, Lon) on the surface of a sphere
    # with the WGS84 datum. Open Street Map and Google Earth use 4326.
    geographies[entity[1]]= geographies[entity[1]].to_crs(epsg = 4326)

    count = len(geographies[entity[1]])
    print('US Census Geographic Entity:',entity[1], '\t count = ',count,'\t\t state = ',entity[0])

US Census Geographic Entity: nation 	 count =  1 		 state =  us
US Census Geographic Entity: region 	 count =  4 		 state =  us
US Census Geographic Entity: division 	 count =  9 		 state =  us
US Census Geographic Entity: state 	 count =  56 		 state =  us
US Census Geographic Entity: county 	 count =  3233 		 state =  us
US Census Geographic Entity: cbsa 	 count =  938 		 state =  us
US Census Geographic Entity: aiannh 	 count =  695 		 state =  us
US Census Geographic Entity: ua10 	 count =  3601 		 state =  us
US Census Geographic Entity: place 	 count =  1746 		 state =  48
US Census Geographic Entity: tract 	 count =  5254 		 state =  48
US Census Geographic Entity: bg 	 count =  15800 		 state =  48


In [ ]:
def create_column(entity):
  column = geographies[entity][["areasqkm"]].describe()
  column = column.rename(columns = {"areasqkm" : entity})
  column.loc["Total"] = geographies[entity]["areasqkm"].sum()

  column.loc["% Total"] = geographies[entity]["areasqkm"].sum() / geographies['nation']["areasqkm"].sum() * 100

  return column

columns = {}
for entity in entities:
  columns[entity[1]] = create_column(entity[1])

In [ ]:
table1 = pd.concat([columns['nation'], 
                    columns['region'],
                    columns['division'],
                    columns['state'],
                    columns['county'],
                    columns['cbsa'],
                    columns['aiannh']], axis=1)
varformat = "{:,.0f}"
table_title = "Table 1. Descriptive statistics for approximate area (sq km) for US Census Geographies."
table1 = table1.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table1

,nation,region,division,state,county,cbsa,aiannh
count,1,4,9,56,"3,233",938,695
mean,"9,366,495","2,338,902","1,039,512","167,259","2,897","4,861",715
std,nan,"1,726,759","812,005","223,451","9,423","6,339","3,430"
min,"9,366,495","436,471","171,789",177,4,93,0
25%,"9,366,495","1,597,533","471,361","26,434","1,104","1,582",3
50%,"9,366,495","2,149,972","714,218","131,626","1,600","2,643",22
75%,"9,366,495","2,891,341","1,341,129","203,523","2,415","5,599",122
max,"9,366,495","4,619,194","2,382,579","1,528,928","383,024","71,008","62,566"
Total,"9,366,495","9,355,608","9,355,608","9,366,495","9,366,495","4,559,404","497,171"
% Total,100,100,100,100,100,49,5


In [ ]:
table2 = pd.concat([columns['tract'],
                    columns['bg'],
                    columns['place']], axis=1)
varformat = "{:,.0f}"
table_title = "Table 2. Descriptive statistics for approximate area (sq km) for Texas Census Geographies."
table2 = table2.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table2


,tract,bg,place
count,"5,254","15,800","1,746"
mean,131,44,20
std,547,266,75
min,0,0,0
25%,2,1,3
50%,5,1,6
75%,28,6,14
max,"16,016","9,525","1,740"
Total,"687,749","687,749","35,734"
% Total,7,7,0


In [ ]:
# Create a guide for the map styles for each entity
# Colors from Color Brewer
# https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=5

maplayers = [['nation','region'],
             ['region','division'],
             ['region','state'],
             ['state','county'],
             ['nation','cbsa'],
             ['nation','aiannh'],
             ['state','place'],
             ['county','tract']]

# Make directory to save output
if not os.path.exists('output_maplayers'):
    os.mkdir('output_maplayers')

In [ ]:
style_function1 = lambda x: {
            'fillColor': 'transparent',
            'color': 'red',
            'weight': 4,
            'fillOpacity': 0
        }
style_function2 = lambda x: {
            'fillColor': 'green',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.25
        }

highlight_function1 = lambda x: {'fillOpacity': 0.5}

maps = {}
for layer in maplayers:
    entity1 = layer[0]
    entity2 = layer[1]
    mapname = entity1+entity2
    print(mapname)

    # What location should the map be centered on?
    center_x = (geographies[entity2].bounds.minx.mean() + geographies[entity2].bounds.maxx.mean())/2
    center_y = (geographies[entity2].bounds.miny.mean() + geographies[entity2].bounds.maxy.mean())/2
    print(f'The center of the {entity2} data file is located at {center_x} {center_y}')

    maps[mapname] = fm.Map(location=[center_y, center_x], 
        max_bounds = True,
        control_scale=True,
        prefer_canvas=True)

    fm.GeoJson(
            geographies[entity1].to_json(),
            name= entity1,
            style_function= style_function1,
            highlight_function=False,
            tooltip=fm.features.GeoJsonTooltip(fields=['NAME','AFFGEOID','GEOID'], sticky=True)                
        ).add_to(maps[mapname])

    fm.GeoJson( 
            geographies[entity2].to_json(),
            name= entity2,
            style_function=style_function2,
            highlight_function=highlight_function1,
            tooltip=fm.features.GeoJsonTooltip(fields=['NAME','AFFGEOID','GEOID'], sticky=False) 
        ).add_to(maps[mapname] )

    fm.LayerControl(collapsed=False, autoZIndex=False).add_to(maps[mapname])

    # How should the map be bound - look for the southwest and northeast corners of the data
    sw_corner = [geographies[entity2].bounds.miny.min(),geographies[entity2].bounds.minx.min()]
    ne_corner = [geographies[entity2].bounds.maxy.max(),geographies[entity2].bounds.maxx.max()]
    print(f'The {entity2} data file is bounded by at {sw_corner} {ne_corner}')
    maps[mapname].fit_bounds([sw_corner, ne_corner])

    maps[mapname].save(f'output_maplayers/{mapname}.html')
    #files.download(f'output_maplayers/{mapname}.html')

nationregion
The center of the region data file is located at -64.13868232500144 40.89900360565585
The region data file is bounded by at [18.917461314593385, -179.14733999999999] [71.35256099911636, 179.77847011250077]
regiondivision
The center of the division data file is located at -78.4711968359081 39.49159054260613
The division data file is bounded by at [18.917461314593385, -179.14733999999999] [71.35256099911636, 179.77847011250077]
regionstate
The center of the state data file is located at -82.68060043007272 36.952816396775916
The state data file is bounded by at [-14.552548987193942, -179.14733999999999] [71.35256099911636, 179.77847011250077]
statecounty
The center of the county data file is located at -91.33944886914709 37.830518833949625
The county data file is bounded by at [-14.552548987193942, -179.14733999999999] [71.35256099911636, 179.77847011250077]
nationcbsa
The center of the cbsa data file is located at -92.35587211232635 37.884555963079656
The cbsa data file is b

In [ ]:
display(maps['nationaiannh'])